In [35]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
from datetime import datetime
import time

# Hacemos una función para extraer todos los URLS y aprovechamos para extraer de estos enlaces los nombres de las empresas:
def extraer_urls(num_paginas=None):
    base_url = "https://www.tecnoempleo.com/ofertas-trabajo/?pagina={}"
    todos_urls = []
    nombres_empresas = [] 
    ciudades = []
    pagina = 1

    while True:
        url = base_url.format(pagina)
        response = requests.get(url)

        if response.status_code != 200:
            print(f"Error al acceder a la página {pagina}")
            break

        soup = BeautifulSoup(response.text, "html.parser")
        enlaces = soup.find_all("a", class_="font-weight-bold text-cyan-700")
        urls = [enlace.get("href") for enlace in enlaces if enlace.get("href")]

        if not urls:
            print(f"No se encontraron más ofertas en la página {pagina}. Finalizando.")
            break

        # Extraemos los nombres de las empresas
        nombre_empresas = soup.find_all("a", class_="text-primary link-muted")
        for empresa in nombre_empresas:
            nombres_empresas.append(empresa.text.strip())

        # Extraemos la ciudad:
        for div in soup.find_all('div', class_='col-12 col-lg-3 text-gray-700 pt-2 text-right hidden-md-down'):
            if div.b:
                ciudades.append(div.b.text.strip())
            else:
                ciudades.append("No especificado")
                
        todos_urls.extend(urls)
        print(f"Página {pagina}: {len(urls)} URLs añadidos.") # Hacemos print para asegurarnos de que está funcionando correctamente.

        # Calculamos el número total de páginas para asegurarnos de que las recorra todas:
        if num_paginas is None:
            total_ofertas = int(soup.find("h1").text.split()[0].replace(".", ""))
            total_paginas = (total_ofertas // 30) + (1 if total_ofertas % 30 != 0 else 0)
            if pagina >= total_paginas:
                break
        elif pagina >= num_paginas:
            break

        pagina += 1

    return todos_urls, nombres_empresas, ciudades

# Hacemos una función que recorre los urls para extraer los datos:
def extraer_datos_de_urls(urls, nombres_empresas, ciudades):
    todas_las_ofertas = []
    todas_las_tecnologias = set()

    # Creamos un bucle que recorra las urls:
    for idx, link in enumerate(urls):
        response_link = requests.get(link)
        if response_link.status_code != 200:
            print(f"Error al acceder a {link}")
            continue

        soup_link = BeautifulSoup(response_link.text, "html.parser")

        # Sacamos título, fecha, habilidades y añadimos el timestamp para que se indique la fecha en la que se han solicitado los datos. 
        titulo = soup_link.find("h1").get_text(strip=True) if soup_link.find("h1") else "Sin título"
        fecha = soup_link.find("span", class_="ml-4").get_text(strip=True) if soup_link.find("span", class_="ml-4") else "Sin fecha"
        habilidades = [x.text.strip() for x in soup_link.find_all("div", class_="d-flex py-2")[1].find_all("a")]
        timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")

        # Tecnologías. En el caso de tecnologías, limpiamos ya la información que sacamos para que nos muestre solola tecnología sin el texto que no es necesario: "Ofertas de Empleo de":
        div = soup_link.find("div", class_="pl--12 pr--12")
        tecnologias = [a["title"].replace("Ofertas de Empleo de ", "").lower() for a in div.find_all("a") if div and a.has_attr("title")]
        todas_las_tecnologias.update(tecnologias)

        # Extraemos todos los datos principales
        todos_textos_enunciados = [span.get_text(strip=True) for span in soup_link.find_all("span", class_="d-inline-block px-2")]
        todos_textos = [span.get_text(strip=True) for span in soup_link.find_all("span", class_="float-end")]
        diccionario_principales = dict(zip(todos_textos_enunciados, todos_textos))

        # Extraemos otros detalles de la oferta
        todos_detalles = soup_link.find_all("p", class_="m-0")
        todos_textos_detalles = [x.text.strip() for x in todos_detalles if len(x["class"]) == 1][1:]
        diccionario_detalles = {}
        for item in todos_textos_detalles:
            if ":" in item:
                clave, valor = item.split(":", 1)
                diccionario_detalles[clave.strip()] = valor.strip()
            else:
                diccionario_detalles[item.strip()] = None

        # Creamos el diccionario para añadir toda la información
        oferta = {
            "Título": titulo,
            "URL": link,
            "Tecnologías": tecnologias,
            "Time Stamp": timestamp,
            "Fecha publicacion": fecha,
            "Habilidades": habilidades,
            "Nombre de empresa": nombres_empresas[idx] if idx < len(nombres_empresas) else "No encontrado",
            "Ciudad": ciudades[idx] if idx < len(ciudades) else "No especificado",  # # Nos aseguramos de que los nombres de empresas y las ciudades se corresponden con los urls sacados
        }

        # Creamos un bucle para asegurarnos de que añadimos todos los datos y, en el caso de que no haya datos, que se añadan también. 
        for clave, valor in diccionario_principales.items():
            oferta[clave] = valor if valor else "No especificado"

        for clave, valor in diccionario_detalles.items():
            oferta[clave] = valor if valor else "No especificado"

        todas_las_ofertas.append(oferta)
        print(f"Procesada oferta: {titulo}")
        print(len(todas_las_ofertas))

    # Creamos el DF
    df_ofertas = pd.DataFrame(todas_las_ofertas)

    # Creamos columnas binarias para cada tecnología. Esto nos facilita hacer un DF para más adelante relacionarlo con el DF principal. 
    for tecnologia in todas_las_tecnologias:
        df_ofertas[tecnologia] = df_ofertas["Tecnologías"].apply(lambda x: 1 if tecnologia in x else 0)
    df_ofertas.drop(columns=["Tecnologías"], inplace=True)

    # Guardamos el CSV
    df_ofertas.to_csv("ofertas_tecnoempleo.csv", index=False, encoding="utf-8-sig") # Usamos el encoding para evitar cualquier error de compatibilidad al abrir el archivo con otros programas. 
    return df_ofertas

# Llamamos a las funciones
urls, nombres_empresas, ciudades = extraer_urls()
df_ofertas = extraer_datos_de_urls(urls, nombres_empresas, ciudades)
print(df_ofertas)

Página 1: 30 URLs añadidos.
Página 2: 30 URLs añadidos.
Página 3: 30 URLs añadidos.
Página 4: 30 URLs añadidos.
Página 5: 30 URLs añadidos.
Página 6: 30 URLs añadidos.
Página 7: 30 URLs añadidos.
Página 8: 30 URLs añadidos.
Página 9: 30 URLs añadidos.
Página 10: 30 URLs añadidos.
Página 11: 30 URLs añadidos.
Página 12: 30 URLs añadidos.
Página 13: 30 URLs añadidos.
Página 14: 30 URLs añadidos.
Página 15: 30 URLs añadidos.
Página 16: 30 URLs añadidos.
Página 17: 30 URLs añadidos.
Página 18: 30 URLs añadidos.
Página 19: 30 URLs añadidos.
Página 20: 30 URLs añadidos.
Página 21: 30 URLs añadidos.
Página 22: 30 URLs añadidos.
Página 23: 30 URLs añadidos.
Página 24: 30 URLs añadidos.
Página 25: 30 URLs añadidos.
Página 26: 30 URLs añadidos.
Página 27: 30 URLs añadidos.
Página 28: 30 URLs añadidos.
Página 29: 30 URLs añadidos.
Página 30: 30 URLs añadidos.
Página 31: 30 URLs añadidos.
Página 32: 30 URLs añadidos.
Página 33: 30 URLs añadidos.
Página 34: 30 URLs añadidos.
Página 35: 30 URLs añad

C:\Users\raque\AppData\Local\Temp\ipykernel_41064\2860599975.py:128: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_ofertas[tecnologia] = df_ofertas["Tecnologías"].apply(lambda x: 1 if tecnologia in x else 0)
C:\Users\raque\AppData\Local\Temp\ipykernel_41064\2860599975.py:128: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_ofertas[tecnologia] = df_ofertas["Tecnologías"].apply(lambda x: 1 if tecnologia in x else 0)
C:\Users\raque\AppData\Local\Temp\ipykernel_41064\2860599975.py:128: PerformanceWarning: DataFrame is highly fra

                                            Título  \
0      UrgenteArquitecto Soluciones Observabilidad   
1                  UrgenteData Analyst BI - Junior   
2                        UrgenteIT Project Manager   
3                      UrgenteSysAdmin/XOps Junior   
4     UrgenteIT System Administrator/Web Developer   
...                                            ...   
3599                  Administrador Sistemas Linux   
3600                          Desarrolladores Java   
3601                          Arquitecto Fullstack   
3602                                Spark Engineer   
3603                           Arquitecto Big Data   

                                                    URL           Time Stamp  \
0     https://www.tecnoempleo.com/arquitecto-solucio...  2025-02-22 17:03:04   
1     https://www.tecnoempleo.com/data-analyst-bi-ju...  2025-02-22 17:03:05   
2     https://www.tecnoempleo.com/it-project-manager...  2025-02-22 17:03:06   
3     https://www.tecnoempleo.c

In [39]:
#Código para ver el df completo 
pd.set_option("display.max_columns", None)
df_ofertas

Título  \
0      UrgenteArquitecto Soluciones Observabilidad   
1                  UrgenteData Analyst BI - Junior   
2                        UrgenteIT Project Manager   
3                      UrgenteSysAdmin/XOps Junior   
4     UrgenteIT System Administrator/Web Developer   
...                                            ...   
3599                  Administrador Sistemas Linux   
3600                          Desarrolladores Java   
3601                          Arquitecto Fullstack   
3602                                Spark Engineer   
3603                           Arquitecto Big Data   

                                                    URL           Time Stamp  \
0     https://www.tecnoempleo.com/arquitecto-solucio...  2025-02-22 17:03:04   
1     https://www.tecnoempleo.com/data-analyst-bi-ju...  2025-02-22 17:03:05   
2     https://www.tecnoempleo.com/it-project-manager...  2025-02-22 17:03:06   
3     https://www.tecnoempleo.com/sysadmin-xops-juni...  2025-02-22 17:03:07   
4     https://www.tecnoempleo.com/it-system-administ...  2025-02-22 17:03:07   
...                                                 ...                  ...   
3599  https://www.tecnoempleo.com/administrador-sist...  2025-02-22 17:51:38   
3600  https://www.tecnoempleo.com/desarrolladores-ja...  2025-02-22 17:51:39   
3601  https://www.tecnoempleo.com/arquitecto-fullsta...  2025-02-22 17:51:40   
3602  https://www.tecnoempleo.com/spark-engineer-mad...  2025-02-22 17:51:40   
3603  https://www.tecnoempleo.com/arquitecto-big-dat...  2025-02-22 17:51:41   

     Fecha publicacion                                        Habilidades  \
0           21/02/2025              [Arquitecto TIC, Técnico de Sistemas]   
1           21/02/2025                               [Analista, Big Data]   
2           21/02/2025  [Arquitecto TIC, Inteligencia Artificial/Machi...   
3           21/02/2025    [Base de Datos/DBA, Redes, Técnico de Sistemas]   
4           21/02/2025  [Administrador, Programador, Técnico de Sistemas]   
...                ...                                                ...   
3599        16/05/2018                                    [Administrador]   
3600        08/11/2017  [Analista Programador, Desarrollador Web, Prog...   
3601        25/10/2017                                   [Arquitecto TIC]   
3602        03/10/2017   [Arquitecto TIC, Desarrollador Web, Programador]   
3603        20/09/2017   [Arquitecto TIC, Desarrollador Web, Programador]   

         Nombre de empresa       Ciudad                                    \
0                NPR Spain  100% remoto                    100% En remoto   
1          Infinity Neural    Barcelona      Badalona -Barcelona(Híbrido)   
2     Talentea Recruitment       Lleida    Mollerussa -Lleida(Presencial)   
3             Beskar Forge       Madrid                   Madrid(Híbrido)   
4              DAC.digital  100% remoto                    100% En remoto   
...                    ...          ...                               ...   
3599          Michael Page       Madrid  Pozuelo De Alarcón -MadridEspaña   
3600          Michael Page       Madrid                    Madrid- España   
3601          Michael Page       Madrid                    Madrid- España   
3602          Michael Page       Madrid           Pozuelo -Madrid- España   
3603          Michael Page       Madrid           Pozuelo -Madrid- España   

     Imprescindible Residir             Funciones           Jornada  \
0                    España        Arquitecto TIC  Jornada completa   
1          Provincia Puesto              Analista  Jornada completa   
2          Provincia Puesto        Arquitecto TIC  Jornada completa   
3                       NaN     Base de Datos/DBA  Jornada completa   
4              Not Required         Administrador  Jornada completa   
...                     ...                   ...               ...   
3599       Provincia Puesto         Administrador  Intensiva Mañana   
3600            

In [37]:
df_ofertas["Ciudad"].unique()

array(['100% remoto', 'Barcelona', 'Lleida', 'Madrid', 'Málaga',
       'Valencia', 'No especificado', 'Baleares', 'Alicante', 'Sevilla',
       'A Coruña', 'Bizkaia', 'Navarra', 'London',
       'Sta. Cruz De Tenerife', 'Zaragoza', 'Huelva', 'Polonia',
       'Gipuzkoa', 'Castellón', 'Valladolid', 'Pontevedra', 'Salamanca',
       'Chipre', 'Albacete', 'Asturias', 'Murcia', 'Álava', 'Melilla',
       'Girona', 'Rep. De Macedonia', 'Bulgaria', 'Almería', 'La Rioja',
       'Zamora', 'Andorra La Vella', 'Córdoba', 'Malta', 'Badajoz',
       'Ourense', 'Tarragona', 'Burgos', 'Granada', 'Teruel', 'Toledo',
       'Cantabria', 'Ciudad Real', 'Jaén', 'Segovia',
       'Las Palmas De Gran Canaria', 'Lugo', 'Bayern', 'Oporto', 'Huesca',
       'Palencia', 'León', 'Guadalajara', 'Ceuta', 'Bruselas'],
      dtype=object)